In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from hyperopt import fmin, tpe, hp
import ipynb.fs.full.utils as utils
import ipynb.fs.full.features as features
import ipynb.fs.full.features_distancias as f_distancias
# import ipynb.fs.full.featuresNLTK as f_nltk

df_train = pd.read_csv('./data/train_filtrado.csv')
# Para usarse con el submit a Kaggle
df_test = pd.read_csv('./data/test.csv')

df_train = features.llenar_nulls(df_train)
df_test = features.llenar_nulls(df_test, hgb_mean=True, df_fill=df_train)

# df_train, df_test = features_de_csvs(df_train, df_test)

In [2]:
df_test_f = features.features_independientes_precio(df_test)
df_test_f = features.features_dependientes_precio(df_test_f, df_train)

df_train_f = features.features_independientes_precio(df_train)
df_train_f = features.features_dependientes_precio(df_train_f, df_train)

df_test_f, cols_tipodepropiedad_ohe = features.columna_a_ohe(df_test_f, 'tipodepropiedad', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_provincia_ohe = features.columna_a_ohe(df_test_f, 'provincia', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_zona_ohe = features.columna_a_ohe(df_test_f, 'zona', df_aux=df_train_f, devolver_cols=True)

df_train_f = features.columna_a_ohe(df_train_f, 'tipodepropiedad', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'provincia', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'zona', df_aux=df_test_f)


df_train_f['fecha'] = pd.to_datetime(df_train_f['fecha']).astype(int)
df_test_f['fecha'] = pd.to_datetime(df_test_f['fecha']).astype(int)

df_train_idf = pd.read_csv('./data/train_idf.csv')
df_test_idf = pd.read_csv('./data/test_idf.csv')

df_train_f = pd.merge(df_train_f, df_train_idf, on= 'id', how= 'left')
df_test_f = pd.merge(df_test_f, df_test_idf, on= 'id', how= 'left')

df_train_f = f_distancias.feature_distancias(df_train_f)
df_test_f = f_distancias.feature_distancias(df_test_f, df_train_f)

# df_train_f = features.KD_feature(df_train_f)
# df_test_f =  features.KD_feature(df_test_f)

In [3]:
# df_train_f, df_test_f = f_nltk.mas_frecuentes_caros_baratos_ohe(df_train_f, 'titulo', 25, 500, df_test,False)
# df_train_f, df_test_f = f_nltk.mas_frecuentes_caros_baratos_ohe(df_train_f, 'descripcion', 50, 500, df_test,True)
# df_train_f, df_test_f = f_nltk.mas_frecuentes_caros_baratos_ohe(df_train_f, 'descripcion', 25, 500, df_test,False)

In [4]:
df_clustering_train = pd.read_csv('data/clustering_train.csv')
df_clustering_test = pd.read_csv('data/clustering_test.csv')

def agregar_clustering(df, train=True):
    df_cluster = df_clustering_train if train else df_clustering_test
    df_merge = pd.merge(df, df_cluster, on='id')
    return df_merge
    


df_train_f = agregar_clustering(df_train_f, True)
df_test_f = agregar_clustering(df_test_f, False)


In [7]:
train_DK = pd.read_csv('./data/df_train_KD.csv')
test_DK = pd.read_csv('./data/df_test_KD.csv')

titulo_caro_train = pd.read_csv('./data/titulo_caro_train.csv')
titulo_caro_test = pd.read_csv('./data/titulo_caro_test.csv')
titulo_barato_train = pd.read_csv('./data/titulo_barato_train.csv')
titulo_barato_test = pd.read_csv('./data/titulo_barato_test.csv')
descripcion_caro_train = pd.read_csv('./data/descripcion_caro_train.csv')
descripcion_caro_test = pd.read_csv('./data/descripcion_caro_test.csv')
descripcion_barato_train = pd.read_csv('./data/descripcion_barato_train.csv')
descripcion_barato_test = pd.read_csv('./data/descripcion_barato_test.csv')

In [8]:
df_train_f = pd.merge(df_train_f, train_DK, on= 'id', how= 'left')
df_test_f = pd.merge(df_test_f, test_DK, on= 'id', how= 'left')

df_train_f = pd.merge(df_train_f, titulo_caro_train, on= 'id', how= 'left')
df_test_f = pd.merge(df_test_f, titulo_caro_test, on= 'id', how= 'left')

df_train_f = pd.merge(df_train_f, titulo_barato_train, on= 'id', how= 'left')
df_test_f = pd.merge(df_test_f, titulo_barato_test, on= 'id', how= 'left')

df_train_f = pd.merge(df_train_f, descripcion_caro_train, on= 'id', how= 'left')
df_test_f = pd.merge(df_test_f, descripcion_caro_test, on= 'id', how= 'left')

df_train_f = pd.merge(df_train_f, descripcion_barato_train, on= 'id', how= 'left')
df_test_f = pd.merge(df_test_f, descripcion_barato_test, on= 'id', how= 'left')

In [9]:
df_train_f = features.rankings(df_train_f, df_train_f)
df_test_f = features.rankings(df_test_f, df_train_f)

In [10]:
from sklearn.model_selection import train_test_split

class LightGBMWrapper(lgb.LGBMRegressor):
    
    def fit(self, x, y):        
        x_train, x_test, y_train, y_test = train_test_split(x, y)
        return super(LightGBMWrapper, self).fit(x_train, y_train)
    
    def predict(self, X):
        return super(LightGBMWrapper, self).predict(X, 
               num_iteration=self.best_iteration_)

hps = {'bagging_fraction': 0.806451877022587,
 'bagging_freq': 62.0,
 'feature_fraction': 0.5379925983440028,
 'learning_rate': 0.1363027714646826,
 'max_depth': 11.0,
 'num_leaves': 113.0,
 'test_size': 0.09575190901892519}

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae', # Si se deja vacio se toma el ideal para llegar al 'objective'
    'num_leaves': int(hps['num_leaves']),
    'learning_rate': hps['learning_rate'],
    'feature_fraction': hps['feature_fraction'],
    'bagging_fraction': hps['bagging_fraction'],
    'bagging_freq': int(hps['bagging_freq']),
    'max_depth': int(hps['max_depth']),
    'verbose': 0
}

lgb_m = LightGBMWrapper(**params)

In [11]:
list(df_train_f.columns)

['id',
 'titulo',
 'descripcion',
 'tipodepropiedad',
 'direccion',
 'ciudad',
 'provincia',
 'antiguedad',
 'habitaciones',
 'garages',
 'banos',
 'metroscubiertos',
 'metrostotales',
 'idzona',
 'lat',
 'lng',
 'fecha',
 'gimnasio',
 'usosmultiples',
 'piscina',
 'escuelascercanas',
 'centroscomercialescercanos',
 'precio',
 'metrostotales_log',
 'metroscubiertos_log',
 'porcentaje_metros',
 'diferencia_metros',
 'intervalo_metros_totales',
 'intervalo_metros_cubiertos',
 'metroscubiertos_bins_unif',
 'metroscubiertos_bins_perc',
 'metros_totales_normalizados',
 'metros_cubiertos_normalizados',
 'escomercial',
 'promedio_metros_tipo_propiedad',
 'promedio_metros_cub_tipo_propiedad',
 'tipo_propiedad_compartida',
 'prop_frecuente',
 'zona',
 'top_provincia',
 'es_ciudad_centrica',
 'promedio_metros_totales_provincia',
 'promedio_metros_cubiertos_provincia',
 'anio',
 'mes',
 'dia',
 'trimestre',
 'dias_desde_datos',
 'meses_desde_datos',
 'dolar_promedio_mes',
 'dolar_promedio_ult_mes

## SelectKBest

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_regression

features = ['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales', 'lat', 'lng', 'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos', 'precio', 'metrostotales_log', 'metroscubiertos_log', 'porcentaje_metros', 'diferencia_metros', 'escomercial', 'promedio_metros_tipo_propiedad', 'promedio_metros_cub_tipo_propiedad', 'tipo_propiedad_compartida', 'prop_frecuente', 'top_provincia', 'es_ciudad_centrica', 'promedio_metros_totales_provincia', 'promedio_metros_cubiertos_provincia', 'anio', 'mes', 'dia', 'trimestre', 'dias_desde_datos', 'meses_desde_datos', 'delincuencia', 'turismo', 'es_antigua', 'cantidad_inquilinos', 'tam_ambientes', 'metros_totales_normalizados', 'metros_cubiertos_normalizados', 'promedio_precio_provincia', 'promedio_precio_ciudad', 'promedio_precio_ciudad_gen', 'varianza_precio_ciudad', 'count_ciudad', 'promedio_id_zona', 'promedio_id_zona_log', 'promedio_id_zona_gen', 'varianza_id_zona', 'count_id_zona', 'promedio_precio_tipo_propiedad', 'promedio_precio_tipo_propiedad_ciudad', 'promedio_precio_tipo_propiedad_ciudad_gen', 'count_tipo_propiedad', 'count_tipo_propiedad_ciudad', 'promedio_por_mes', 'varianza_por_mes', 'promedio_precio_habitaciones', 'promedio_precio_habitaciones_banos_garages', 'promedio_precio_banos_garages', 'promedio_precio_hbg_tipo_propiedad', 'promedio_precio_hbg_tipo_propiedad_provincia', 'lat_norm', 'lng_norm', 'promedio_precio_booleanos', 'puntaje', 'tipodepropiedad_Apartamento', 'tipodepropiedad_Bodega comercial', 'tipodepropiedad_Casa', 'tipodepropiedad_Casa en condominio', 'tipodepropiedad_Casa uso de suelo', 'tipodepropiedad_Departamento Compartido', 'tipodepropiedad_Duplex', 'tipodepropiedad_Edificio', 'tipodepropiedad_Inmuebles productivos urbanos', 'tipodepropiedad_Local Comercial', 'tipodepropiedad_Local en centro comercial', 'tipodepropiedad_Oficina comercial', 'tipodepropiedad_Otros', 'tipodepropiedad_Quinta Vacacional', 'tipodepropiedad_Rancho', 'tipodepropiedad_Terreno', 'tipodepropiedad_Terreno comercial', 'tipodepropiedad_Villa', 'tipodepropiedad_otro', 'provincia_Aguascalientes', 'provincia_Baja California Norte', 'provincia_Baja California Sur', 'provincia_Campeche', 'provincia_Chiapas', 'provincia_Chihuahua', 'provincia_Coahuila', 'provincia_Colima', 'provincia_Distrito Federal', 'provincia_Durango', 'provincia_Edo. de Mxico', 'provincia_Guanajuato', 'provincia_Guerrero', 'provincia_Hidalgo', 'provincia_Jalisco', 'provincia_Michoacn', 'provincia_Morelos', 'provincia_Nayarit', 'provincia_Nuevo Len', 'provincia_Oaxaca', 'provincia_Puebla', 'provincia_Quertaro', 'provincia_Quintana Roo', 'provincia_San luis Potos', 'provincia_Sinaloa', 'provincia_Sonora', 'provincia_Tabasco', 'provincia_Tamaulipas', 'provincia_Tlaxcala', 'provincia_Veracruz', 'provincia_Yucatn', 'provincia_otro', 'zona_Centro', 'zona_Norte ', 'zona_Sur'] 

x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_f, 'precio', features, test_size=0.001)

selector_kbest = SelectKBest(f_regression, k=30)
selector_kbest.fit_transform(x_train, y_train)
x_train.columns[selector_kbest.get_support()]

In [ ]:
# Me mata la RAM, SWAP y me deja triste
# x_train_g = SelectKBest(mutual_info_regression, k=20).fit_transform(x_train_f, y_train)

## SelectFromModel

In [ ]:
from sklearn.feature_selection import SelectFromModel

lgb_m = LightGBMWrapper(**params)
lgb_m.fit(x_train, y_train)

model = SelectFromModel(lgb_m, prefit=True)
x_train.columns[model.get_support()]

In [ ]:
from sklearn.feature_selection import SelectFromModel

features_sub = ['antiguedad', 'metroscubiertos', 'metrostotales', 
       'porcentaje_metros', 'diferencia_metros', 'mes', 'dia', 'dias_desde_datos',
       'meses_desde_datos', 'tam_ambientes', 'metros_totales_normalizados',
       'metros_cubiertos_normalizados', 'promedio_precio_ciudad',
       'varianza_precio_ciudad', 'count_ciudad', 'promedio_id_zona',
       'count_id_zona', 'promedio_precio_tipo_propiedad_ciudad_gen',
       'promedio_por_mes',  'promedio_precio_hbg_tipo_propiedad',
       'lat_norm', 'lng_norm', 'puntaje']

lgb_m = LightGBMWrapper(**params)
lgb_m.fit(x_train[features_sub], y_train)

model = SelectFromModel(lgb_m, prefit=True)
x_train[features_sub].columns[model.get_support()]

## Recursive Feature Elimination (RFE)

In [13]:
from sklearn.feature_selection import RFE

features = ['antiguedad',
 'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales', 'lat', 'lng', 'fecha',
 'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos', 'precio',
 'metrostotales_log', 'metroscubiertos_log', 'porcentaje_metros', 'diferencia_metros', 'metroscubiertos_bins_unif', 'metroscubiertos_bins_perc',
 'metros_totales_normalizados', 'metros_cubiertos_normalizados', 'escomercial',
'tipo_propiedad_compartida', 'prop_frecuente', 'top_provincia', 'es_ciudad_centrica', 
 'anio', 'mes', 'dia', 'trimestre', 'dias_desde_datos', 'meses_desde_datos', 
 'delincuencia', 'turismo', 'es_antigua', 'antiguedad_bins_unif', 'antiguedad_bins_perc',
 'cantidad_inquilinos', 'tam_ambientes','promedio_precio_provincia',  'promedio_precio_ciudad',
    'count_ciudad', 'promedio_id_zona', 'count_id_zona',
 'promedio_precio_tipo_propiedad', 'count_tipo_propiedad', 'count_tipo_propiedad_ciudad', 'lat_norm', 'lng_norm', 'puntaje', 
 'idf_titulo', 'idf_descripcion', 'peso_titulo', 'peso_descripcion', 'distancia_ciudad_centrica',
 'distancia_centro_mexico','distancia_ciudad_cara', 'kmeans_basicos', 'kmeans_superficie', 'kmeans_metroscubiertos',
 'kmeans_fechas', 'kmeans_latlng',  'similares_count',
 'densidad', 'dolar_promedio_ult_mes', 'dolar_diferencia_ult_mes', 'dolar_diferencia_mes_sum', 'ranking_en_provincia',
 'palabra_polanco',
 'palabra_condominio',
 'palabra_lomas',
 'palabra_excelente',
 'palabra_ciudad',
 'palabra_méxico',
 'palabra_fracc',
 'palabra_terreno',
 'palabra_recamaras_x',
 'palabra_infonavit',
 'palabra_subsidio',
 'palabra_cuarto',
 'palabra_servicio',
 'palabra_vestidor',
 'palabra_nbsp',
 'palabra_recamaras_y',
 'palabra_patio',
 'ranking_en_ciudad',
 'ranking_en_provincia_ciudad_precio',
 'ranking_en_provincia_ciudad_cantidad',
 'ranking_en_provincia_tipodepropiedad_precio',
 'ranking_en_provincia_tipodepropiedad_cantidad',
 'ranking_en_provincia_intervalo_metros_totales_precio',
 'ranking_en_provincia_intervalo_metros_totales_cantidad',
 'ranking_en_provincia_intervalo_metros_cubiertos_precio',
 'ranking_en_provincia_intervalo_metros_cubiertos_cantidad',
 'ranking_en_tipodepropiedad_intervalo_metros_totales_precio',
 'ranking_en_tipodepropiedad_intervalo_metros_totales_cantidad',
 'ranking_en_tipodepropiedad_intervalo_metros_cubiertos_precio',
 'ranking_en_tipodepropiedad_intervalo_metros_cubiertos_cantidad']


x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_f, 'precio', features, test_size=0.001)

lgb_m = LightGBMWrapper(**params)

selector = RFE(lgb_m, n_features_to_select=25, verbose=1)
selector.fit(x_train, y_train)

x_train.columns[selector.get_support()]

Fitting estimator with 99 features.
Fitting estimator with 98 features.
Fitting estimator with 97 features.
Fitting estimator with 96 features.
Fitting estimator with 95 features.
Fitting estimator with 94 features.
Fitting estimator with 93 features.
Fitting estimator with 92 features.
Fitting estimator with 91 features.
Fitting estimator with 90 features.
Fitting estimator with 89 features.
Fitting estimator with 88 features.
Fitting estimator with 87 features.
Fitting estimator with 86 features.
Fitting estimator with 85 features.
Fitting estimator with 84 features.
Fitting estimator with 83 features.
Fitting estimator with 82 features.


KeyboardInterrupt: 

In [ ]:
def full_display(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
        display(df)
        
df_ranks = pd.DataFrame(np.array([selector.ranking_, list(x_train.columns)]).T, columns=['rank', 'col'])
df_ranks['rank'] = df_ranks['rank'].astype(int)
full_display(df_ranks.sort_values('rank'))

df_ranks[df_ranks['rank'] == 1]['col'].tolist()

## Recursive Feature Elimination w/ Cross Validation (RFECV)

In [ ]:
df_train_f[[
 'count_ciudad',
 'promedio_id_zona',
 'promedio_id_zona_log',
 'promedio_id_zona_gen',
 'varianza_id_zona',
 'count_id_zona',
 'promedio_precio_tipo_propiedad',
 'promedio_precio_tipo_propiedad_ciudad',
 'promedio_precio_tipo_propiedad_ciudad_gen',
 'count_tipo_propiedad',
 'count_tipo_propiedad_ciudad',
 'promedio_por_mes',
 'varianza_por_mes',
 'promedio_precio_habitaciones',
 'promedio_precio_habitaciones_banos_garages',
 'promedio_precio_banos_garages',
 'promedio_precio_hbg_tipo_propiedad',
 'promedio_precio_hbg_tipo_propiedad_provincia',
 'promedio_precio_hbg_tipo_propiedad_provincia_gen',
 'lat_norm',
 'lng_norm',
 'promedio_precio_booleanos',
 'puntaje',
 'idf_titulo',
 'idf_descripcion',
 'peso_titulo',
 'peso_descripcion',
 'distancia_ciudad_centrica',
 'distancia_centro_mexico',
 'distancia_ciudad_cara',
 'kmeans_basicos',
 'kmeans_superficie',
 'kmeans_metroscubiertos',
 'kmeans_fechas',
 'kmeans_latlng',
 'palabra_venta',
 'palabra_casa',
 'palabra_departamento',
 'palabra_san',
 'palabra_polanco',
 'palabra_lomas',
 'palabra_valle',
 'palabra_excelente',
 'palabra_condominio',
 'palabra_terreno',
 'palabra_residencia',
 'palabra_edificio',
 'palabra_ciudad',
 'palabra_hermosa',
 'palabra_méxico',
 'palabra_pedro',
 'palabra_jardines',
 'palabra_bosques',
 'palabra_preciosa',
 'palabra_santa',
 'palabra_garza',
 'palabra_fe',
 'palabra_pedregal',
 'palabra_preventa',
 'palabra_colonia',
 'palabra_zona',
 'palabra_comercial',
 'palabra_bosque',
 'palabra_tecamachalco',
 'palabra_nuevo',
 'palabra_montaña',
 'palabra_chapultepec',
 'palabra_depto',
 'palabra_lujo',
 'palabra_vista',
 'palabra_garcia',
 'palabra_miguel',
 'palabra_parque',
 'palabra_calle',
 'palabra_oportunidad',
 'palabra_centro',
 'palabra_club',
 'palabra_departamentos',
 'palabra_cerrada',
 'palabra_uso',
 'palabra_proyecto',
 'palabra_real',
 'palabra_distrito',
 'palabra_cuajimalpa',
 'palabra_av',
 'similares_count',
 'densidad',
 'ranking_en_provincia',
 'ranking_en_ciudad',
 'ranking_en_provincia_ciudad_precio',
 'ranking_en_provincia_ciudad_cantidad',
 'ranking_en_provincia_tipodepropiedad_precio',
 'ranking_en_provincia_tipodepropiedad_cantidad',
 'ranking_en_provincia_intervalo_metros_totales_precio',
 'ranking_en_provincia_intervalo_metros_totales_cantidad',
 'ranking_en_provincia_intervalo_metros_cubiertos_precio',
 'ranking_en_provincia_intervalo_metros_cubiertos_cantidad',
 'ranking_en_tipodepropiedad_intervalo_metros_totales_precio',
 'ranking_en_tipodepropiedad_intervalo_metros_totales_cantidad',
 'ranking_en_tipodepropiedad_intervalo_metros_cubiertos_precio',
 'ranking_en_tipodepropiedad_intervalo_metros_cubiertos_cantidad']].isnull().any()

In [15]:
from sklearn.feature_selection import RFECV

features = ['antiguedad',
 'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales', 'lat', 'lng', 'fecha',
 'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos', 'precio',
 'metrostotales_log', 'metroscubiertos_log', 'porcentaje_metros', 'diferencia_metros', 'metroscubiertos_bins_unif', 'metroscubiertos_bins_perc',
 'metros_totales_normalizados', 'metros_cubiertos_normalizados', 'escomercial',
'tipo_propiedad_compartida', 'prop_frecuente', 'top_provincia', 'es_ciudad_centrica', 
 'anio', 'mes', 'dia', 'trimestre', 'dias_desde_datos', 'meses_desde_datos', 
 'delincuencia', 'turismo', 'es_antigua', 'antiguedad_bins_unif', 'antiguedad_bins_perc',
 'cantidad_inquilinos', 'tam_ambientes','promedio_precio_provincia',  'promedio_precio_ciudad',
    'count_ciudad', 'promedio_id_zona', 'count_id_zona',
 'promedio_precio_tipo_propiedad', 'count_tipo_propiedad', 'count_tipo_propiedad_ciudad', 'lat_norm', 'lng_norm', 'puntaje', 
 'idf_titulo', 'idf_descripcion', 'peso_titulo', 'peso_descripcion', 'distancia_ciudad_centrica',
 'distancia_centro_mexico','distancia_ciudad_cara', 'kmeans_basicos', 'kmeans_superficie', 'kmeans_metroscubiertos',
 'kmeans_fechas', 'kmeans_latlng',  'similares_count',
 'densidad', 'dolar_promedio_ult_mes', 'dolar_diferencia_ult_mes', 'dolar_diferencia_mes_sum', 'ranking_en_provincia',
 'palabra_polanco',
 'palabra_condominio',
 'palabra_lomas',
 'palabra_excelente',
 'palabra_ciudad',
 'palabra_méxico',
 'palabra_fracc',
 'palabra_terreno',
 'palabra_infonavit',
 'palabra_subsidio',
 'palabra_cuarto',
 'palabra_servicio',
 'palabra_vestidor',
 'palabra_nbsp',
 'palabra_patio',
 'ranking_en_ciudad',
 'ranking_en_provincia_ciudad_precio',
 'ranking_en_provincia_ciudad_cantidad',
 'ranking_en_provincia_tipodepropiedad_precio',
 'ranking_en_provincia_tipodepropiedad_cantidad',
 'ranking_en_provincia_intervalo_metros_totales_precio',
 'ranking_en_provincia_intervalo_metros_totales_cantidad',
 'ranking_en_provincia_intervalo_metros_cubiertos_precio',
 'ranking_en_provincia_intervalo_metros_cubiertos_cantidad',
 'ranking_en_tipodepropiedad_intervalo_metros_totales_precio',
 'ranking_en_tipodepropiedad_intervalo_metros_totales_cantidad',
 'ranking_en_tipodepropiedad_intervalo_metros_cubiertos_precio',
 'ranking_en_tipodepropiedad_intervalo_metros_cubiertos_cantidad']

x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_f, 'precio', features, test_size=0.001)

lgb_m = LightGBMWrapper(**params)

selector = RFECV(lgb_m, verbose=1, scoring = 'neg_mean_absolute_error')
selector.fit(x_train, y_train)

x_train.columns[selector.get_support()]

/home/camix/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Fitting estimator with 99 features.
Fitting estimator with 98 features.
Fitting estimator with 97 features.
Fitting estimator with 96 features.
Fitting estimator with 95 features.
Fitting estimator with 94 features.
Fitting estimator with 93 features.
Fitting estimator with 92 features.
Fitting estimator with 91 features.
Fitting estimator with 90 features.
Fitting estimator with 89 features.
Fitting estimator with 88 features.
Fitting estimator with 87 features.
Fitting estimator with 86 features.
Fitting estimator with 85 features.
Fitting estimator with 84 features.
Fitting estimator with 83 features.
Fitting estimator with 82 features.
Fitting estimator with 81 features.
Fitting estimator with 80 features.
Fitting estimator with 79 features.
Fitting estimator with 78 features.
Fitting estimator with 77 features.
Fitting estimator with 76 features.
Fitting estimator with 75 features.
Fitting estimator with 74 features.
Fitting estimator with 73 features.
Fitting estimator with 72 fe

Fitting estimator with 66 features.
Fitting estimator with 65 features.
Fitting estimator with 64 features.
Fitting estimator with 63 features.
Fitting estimator with 62 features.
Fitting estimator with 61 features.
Fitting estimator with 60 features.
Fitting estimator with 59 features.
Fitting estimator with 58 features.
Fitting estimator with 57 features.
Fitting estimator with 56 features.
Fitting estimator with 55 features.
Fitting estimator with 54 features.
Fitting estimator with 53 features.
Fitting estimator with 52 features.
Fitting estimator with 51 features.
Fitting estimator with 50 features.
Fitting estimator with 49 features.
Fitting estimator with 48 features.
Fitting estimator with 47 features.
Fitting estimator with 46 features.
Fitting estimator with 45 features.
Fitting estimator with 44 features.
Fitting estimator with 43 features.
Fitting estimator with 42 features.
Fitting estimator with 41 features.
Fitting estimator with 40 features.
Fitting estimator with 39 fe

Index(['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos',
       'metrostotales', 'lat', 'lng', 'fecha', 'gimnasio', 'usosmultiples',
       'piscina', 'escuelascercanas', 'centroscomercialescercanos',
       'metrostotales_log', 'metroscubiertos_log', 'porcentaje_metros',
       'diferencia_metros', 'metroscubiertos_bins_unif',
       'metroscubiertos_bins_perc', 'metros_totales_normalizados',
       'metros_cubiertos_normalizados', 'escomercial',
       'tipo_propiedad_compartida', 'prop_frecuente', 'top_provincia',
       'es_ciudad_centrica', 'anio', 'mes', 'dia', 'trimestre',
       'dias_desde_datos', 'meses_desde_datos', 'dolar_promedio_ult_mes',
       'dolar_diferencia_ult_mes', 'dolar_diferencia_mes_sum', 'delincuencia',
       'turismo', 'es_antigua', 'antiguedad_bins_unif', 'antiguedad_bins_perc',
       'cantidad_inquilinos', 'tam_ambientes', 'promedio_precio_provincia',
       'promedio_precio_ciudad', 'count_ciudad', 'promedio_id_zona',
       'count_id

In [16]:
def full_display(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
        display(df)
        
df_ranks = pd.DataFrame(np.array([selector.ranking_, list(x_train.columns)]).T, columns=['rank', 'col'])
df_ranks['rank'] = df_ranks['rank'].astype(int)
full_display(df_ranks.sort_values('rank'))

df_ranks[df_ranks['rank'] == 1]['col'].tolist()

,rank,col
0,1,antiguedad
71,1,palabra_excelente
70,1,palabra_lomas
69,1,palabra_condominio
68,1,palabra_polanco
67,1,densidad
66,1,similares_count
65,1,kmeans_latlng
64,1,kmeans_fechas
63,1,kmeans_metroscubiertos


['antiguedad',
 'habitaciones',
 'garages',
 'banos',
 'metroscubiertos',
 'metrostotales',
 'lat',
 'lng',
 'fecha',
 'gimnasio',
 'usosmultiples',
 'piscina',
 'escuelascercanas',
 'centroscomercialescercanos',
 'metrostotales_log',
 'metroscubiertos_log',
 'porcentaje_metros',
 'diferencia_metros',
 'metroscubiertos_bins_unif',
 'metroscubiertos_bins_perc',
 'metros_totales_normalizados',
 'metros_cubiertos_normalizados',
 'escomercial',
 'tipo_propiedad_compartida',
 'prop_frecuente',
 'top_provincia',
 'es_ciudad_centrica',
 'anio',
 'mes',
 'dia',
 'trimestre',
 'dias_desde_datos',
 'meses_desde_datos',
 'dolar_promedio_ult_mes',
 'dolar_diferencia_ult_mes',
 'dolar_diferencia_mes_sum',
 'delincuencia',
 'turismo',
 'es_antigua',
 'antiguedad_bins_unif',
 'antiguedad_bins_perc',
 'cantidad_inquilinos',
 'tam_ambientes',
 'promedio_precio_provincia',
 'promedio_precio_ciudad',
 'count_ciudad',
 'promedio_id_zona',
 'count_id_zona',
 'promedio_precio_tipo_propiedad',
 'count_tipo_p

## PCA

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(0.90)

features = ['metrostotales_log', 'metroscubiertos_log', 'porcentaje_metros', 'diferencia_metros',
 'metroscubiertos_bins_perc', 'escomercial', 'promedio_metros_tipo_propiedad', 'promedio_metros_cub_tipo_propiedad',
 'top_provincia', 'promedio_metros_totales_provincia', 'promedio_metros_cubiertos_provincia',
 'mes', 'dia', 'dias_desde_datos', 'meses_desde_datos', 'antiguedad_bins_unif', 'antiguedad_bins_perc',
 'cantidad_inquilinos', 'tam_ambientes', 'metros_totales_normalizados', 'metros_cubiertos_normalizados',
 'promedio_precio_provincia', 'promedio_precio_ciudad', 'promedio_precio_ciudad_gen', 'varianza_precio_ciudad',
 'count_ciudad', 'promedio_id_zona', 'promedio_id_zona_log', 'promedio_id_zona_gen', 'varianza_id_zona',
 'count_id_zona', 'promedio_precio_tipo_propiedad', 'promedio_precio_tipo_propiedad_ciudad',
 'promedio_precio_tipo_propiedad_ciudad_gen', 'count_tipo_propiedad', 'count_tipo_propiedad_ciudad',
 'promedio_por_mes', 'varianza_por_mes', 'promedio_precio_habitaciones', 'promedio_precio_habitaciones_banos_garages',
 'promedio_precio_banos_garages', 'promedio_precio_hbg_tipo_propiedad',
 'promedio_precio_hbg_tipo_propiedad_provincia', 'lat_norm', 'lng_norm', 'promedio_precio_booleanos',
 'puntaje', 'idf_titulo', 'idf_descripcion', 'peso_descripcion']

df_train_red = pca.fit_transform(utils.filtrar_features(df_train_f, features))

df_train_red.shape

In [ ]:
len(df_train_red.T)

In [ ]:
np.log(df_train_f['precio']).hist()

In [ ]:
def full_display(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        display(df)
        
# full_display(x_train.head(1))